In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14863,2022-01-31T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1496,141,1637,243323,...,DECEDUTI: N. 1 IL 31/01/22 N. 3 IL 30/01/22 - ...,7.0,NaN,Sul numero complessivo dei casi confermati com...,462196.0,161473.0,4183563.0,5595362.0,ITG,ITG1
14864,2022-01-31T17:00:00,ITA,9,Toscana,43.769231,11.255889,1365,112,1477,151308,...,NaN,12.0,NaN,NaN,507022.0,232284.0,6065431.0,5051213.0,ITI,ITI1
14865,2022-01-31T17:00:00,ITA,10,Umbria,43.106758,12.388247,212,11,223,22779,...,- Si fa presente che 7 dei ricoveri non UTI ap...,0.0,NaN,NaN,107728.0,47344.0,1459595.0,1952351.0,ITI,ITI2
14866,2022-01-31T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,64,7,71,4567,...,NaN,1.0,NaN,NaN,14317.0,14800.0,132136.0,302289.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14863,2022-01-31T17:00:00,19,Sicilia,1496,141,1637,243323,244960,2516,3328,...,623669,9778925,4229923.0,7.0,462196.0,161473.0,4183563.0,5595362.0,ITG,ITG1
14864,2022-01-31T17:00:00,9,Toscana,1365,112,1477,151308,152785,-3626,4109,...,739306,11116644,4157229.0,12.0,507022.0,232284.0,6065431.0,5051213.0,ITI,ITI1
14865,2022-01-31T17:00:00,10,Umbria,212,11,223,22779,23002,-186,593,...,155072,3411946,625150.0,0.0,107728.0,47344.0,1459595.0,1952351.0,ITI,ITI2
14866,2022-01-31T17:00:00,2,Valle d'Aosta,64,7,71,4567,4638,-331,56,...,29117,434425,124211.0,1.0,14317.0,14800.0,132136.0,302289.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-01-31', '2022-01-30')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-31T17:00:00,1,Piemonte,2111,127,2238,138003,140241,-8274,6241,...,873864,13869108,3566740.0,7.0,454475.0,419389.0,4510936.0,9358172.0,ITC,ITC1
1,2022-01-31T17:00:00,2,Valle d'Aosta,64,7,71,4567,4638,-331,56,...,29117,434425,124211.0,1.0,14317.0,14800.0,132136.0,302289.0,ITC,ITC2
2,2022-01-31T17:00:00,3,Lombardia,3016,251,3267,368874,372141,-15149,5417,...,2129081,30489791,7589397.0,12.0,1265351.0,863730.0,14439805.0,16049986.0,ITC,ITC4
3,2022-01-31T17:00:00,5,Veneto,1503,157,1660,245853,247513,-3242,4877,...,1151107,23376300,4435581.0,13.0,711776.0,439331.0,8977724.0,14398576.0,ITH,ITH3
4,2022-01-31T17:00:00,6,Friuli Venezia Giulia,519,42,561,61669,62230,-60,1032,...,268017,5295722,1050917.0,4.0,170597.0,97420.0,2904866.0,2390856.0,ITH,ITH4
5,2022-01-31T17:00:00,7,Liguria,727,33,760,46349,47109,0,1720,...,296822,4330636,1179951.0,1.0,200104.0,96718.0,2182646.0,2147990.0,ITC,ITC3
6,2022-01-31T17:00:00,8,Emilia-Romagna,2701,149,2850,369329,372179,-8249,8983,...,1050480,13575984,2576346.0,7.0,786852.0,263628.0,8092044.0,5483940.0,ITH,ITH5
7,2022-01-31T17:00:00,9,Toscana,1365,112,1477,151308,152785,-3626,4109,...,739306,11116644,4157229.0,12.0,507022.0,232284.0,6065431.0,5051213.0,ITI,ITI1
8,2022-01-31T17:00:00,10,Umbria,212,11,223,22779,23002,-186,593,...,155072,3411946,625150.0,0.0,107728.0,47344.0,1459595.0,1952351.0,ITI,ITI2
9,2022-01-31T17:00:00,11,Marche,322,62,384,26566,26950,15,1753,...,260821,2496990,1562916.0,3.0,185307.0,75514.0,1831526.0,665464.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-30T17:00:00,1,Piemonte,2083,125,2208,146307,148515,-2345,6244,...,867623,13799501,3557802.0,10.0,453401.0,414222.0,4504254.0,9295247.0,ITC,ITC1
1,2022-01-30T17:00:00,2,Valle d'Aosta,58,7,65,4904,4969,11,208,...,29061,433782,124062.0,1.0,14324.0,14737.0,132088.0,301694.0,ITC,ITC2
2,2022-01-30T17:00:00,3,Lombardia,3004,254,3258,384032,387290,-14802,14558,...,2123664,30429482,7583289.0,5.0,1263534.0,860130.0,14426604.0,16002878.0,ITC,ITC4
3,2022-01-30T17:00:00,5,Veneto,1480,160,1640,249115,250755,-2612,11233,...,1146230,23328544,4433046.0,8.0,712118.0,434112.0,8970604.0,14357940.0,ITH,ITH3
4,2022-01-30T17:00:00,6,Friuli Venezia Giulia,505,43,548,61742,62290,-4874,2680,...,266990,5286506,1050090.0,4.0,170353.0,96637.0,2902039.0,2384467.0,ITH,ITH4
5,2022-01-30T17:00:00,7,Liguria,713,35,748,46361,47109,-213,3209,...,295102,4317925,1178167.0,3.0,199408.0,95694.0,2179890.0,2138035.0,ITC,ITC3
6,2022-01-30T17:00:00,8,Emilia-Romagna,2632,151,2783,377645,380428,-730,13091,...,1041847,13544323,2573685.0,10.0,781845.0,260002.0,8081442.0,5462881.0,ITH,ITH5
7,2022-01-30T17:00:00,9,Toscana,1315,111,1426,154985,156411,-4682,8404,...,735197,11084693,4150678.0,10.0,504712.0,230485.0,6052552.0,5032141.0,ITI,ITI1
8,2022-01-30T17:00:00,10,Umbria,205,11,216,22972,23188,-893,1619,...,154479,3405735,624534.0,4.0,107578.0,46901.0,1459010.0,1946725.0,ITI,ITI2
9,2022-01-30T17:00:00,11,Marche,305,59,364,26571,26935,-690,4520,...,259068,2483869,1558001.0,0.0,185019.0,74049.0,1822830.0,661039.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-01-31T17:00:00,1,Piemonte,2111,127,2238,138003,140241,-8274,6241,...,13869108,3566740.0,7.0,454475.0,419389.0,4510936.0,9358172.0,ITC,ITC1,4.341375e+06
1,2022-01-31T17:00:00,2,Valle d'Aosta,64,7,71,4567,4638,-331,56,...,434425,124211.0,1.0,14317.0,14800.0,132136.0,302289.0,ITC,ITC2,1.255010e+05
2,2022-01-31T17:00:00,3,Lombardia,3016,251,3267,368874,372141,-15149,5417,...,30489791,7589397.0,12.0,1265351.0,863730.0,14439805.0,16049986.0,ITC,ITC4,1.010397e+07
3,2022-01-31T17:00:00,5,Veneto,1503,157,1660,245853,247513,-3242,4877,...,23376300,4435581.0,13.0,711776.0,439331.0,8977724.0,14398576.0,ITH,ITH3,4.907704e+06
4,2022-01-31T17:00:00,6,Friuli Venezia Giulia,519,42,561,61669,62230,-60,1032,...,5295722,1050917.0,4.0,170597.0,97420.0,2904866.0,2390856.0,ITH,ITH4,1.211357e+06
5,2022-01-31T17:00:00,7,Liguria,727,33,760,46349,47109,0,1720,...,4330636,1179951.0,1.0,200104.0,96718.0,2182646.0,2147990.0,ITC,ITC3,1.543127e+06
6,2022-01-31T17:00:00,8,Emilia-Romagna,2701,149,2850,369329,372179,-8249,8983,...,13575984,2576346.0,7.0,786852.0,263628.0,8092044.0,5483940.0,ITH,ITH5,4.467118e+06
7,2022-01-31T17:00:00,9,Toscana,1365,112,1477,151308,152785,-3626,4109,...,11116644,4157229.0,12.0,507022.0,232284.0,6065431.0,5051213.0,ITI,ITI1,3.722729e+06
8,2022-01-31T17:00:00,10,Umbria,212,11,223,22779,23002,-186,593,...,3411946,625150.0,0.0,107728.0,47344.0,1459595.0,1952351.0,ITI,ITI2,8.802850e+05
9,2022-01-31T17:00:00,11,Marche,322,62,384,26566,26950,15,1753,...,2496990,1562916.0,3.0,185307.0,75514.0,1831526.0,665464.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-01-31T17:00:00,1,Piemonte,2111,127,2238,138003,140241,-8274,6241,...,9358172.0,ITC,ITC1,4.341375e+06,Piemonte,2111,6794,127,628,99
1,2022-01-31T17:00:00,2,Valle d'Aosta,64,7,71,4567,4638,-331,56,...,302289.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,64,163,7,34,0
2,2022-01-31T17:00:00,3,Lombardia,3016,251,3267,368874,372141,-15149,5417,...,16049986.0,ITC,ITC4,1.010397e+07,Lombardia,3016,10457,251,1810,0
3,2022-01-31T17:00:00,5,Veneto,1503,157,1660,245853,247513,-3242,4877,...,14398576.0,ITH,ITH3,4.907704e+06,Veneto,1503,6000,157,1000,0
4,2022-01-31T17:00:00,6,Friuli Venezia Giulia,519,42,561,61669,62230,-60,1032,...,2390856.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,519,1277,42,175,0
5,2022-01-31T17:00:00,7,Liguria,727,33,760,46349,47109,0,1720,...,2147990.0,ITC,ITC3,1.543127e+06,Liguria,727,1824,33,221,12
6,2022-01-31T17:00:00,8,Emilia-Romagna,2701,149,2850,369329,372179,-8249,8983,...,5483940.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,2701,9001,149,889,0
7,2022-01-31T17:00:00,9,Toscana,1365,112,1477,151308,152785,-3626,4109,...,5051213.0,ITI,ITI1,3.722729e+06,Toscana,1365,5033,112,570,8
8,2022-01-31T17:00:00,10,Umbria,212,11,223,22779,23002,-186,593,...,1952351.0,ITI,ITI2,8.802850e+05,Umbria,212,662,11,86,41
9,2022-01-31T17:00:00,11,Marche,322,62,384,26566,26950,15,1753,...,665464.0,ITI,ITI3,1.518400e+06,Marche,322,1027,62,256,49


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-01-31T17:00:00,8,Emilia-Romagna,2701,149,2850,369329,372179,-8249,8983,...,16.76,-8316,-4108,31661,2661.0,36,16846,28.37,0.20109,23.52
1,2022-01-31T17:00:00,12,Lazio,2145,206,2351,284387,286738,-5045,6615,...,21.85,-5046,-4918,58136,4368.0,29,11631,11.38,0.11278,15.04
2,2022-01-31T17:00:00,1,Piemonte,2111,127,2238,138003,140241,-8274,6241,...,20.22,-8304,-3,69607,8938.0,18,14497,8.97,0.14376,20.13
3,2022-01-31T17:00:00,3,Lombardia,3016,251,3267,368874,372141,-15149,5417,...,13.87,-15158,-9141,60309,6108.0,76,20490,8.98,0.05361,21.07
4,2022-01-31T17:00:00,15,Campania,1391,89,1480,225277,226757,-5336,5062,...,10.96,-5324,-4752,42979,6506.0,51,10347,11.78,0.08749,17.82
5,2022-01-31T17:00:00,5,Veneto,1503,157,1660,245853,247513,-3242,4877,...,15.70,-3262,-6356,47756,2535.0,23,8096,10.21,0.09937,23.46
6,2022-01-31T17:00:00,9,Toscana,1365,112,1477,151308,152785,-3626,4109,...,19.65,-3677,-4295,31951,6551.0,27,7708,12.86,0.11038,19.86
7,2022-01-31T17:00:00,19,Sicilia,1496,141,1637,243323,244960,2516,3328,...,16.63,2489,-2813,21906,21716.0,29,1054,15.19,0.06698,12.55
8,2022-01-31T17:00:00,16,Puglia,728,59,787,136248,137035,-3630,2638,...,11.73,-3623,429,27689,2929.0,17,6251,9.53,0.06581,15.03
9,2022-01-31T17:00:00,11,Marche,322,62,384,26566,26950,15,1753,...,24.22,-5,-2767,13121,4915.0,6,1732,13.36,0.11545,17.18


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,24,66,90,26.67
1,Marche,62,194,256,24.22
2,Friuli Venezia Giulia,42,133,175,24.00
3,Lazio,206,737,943,21.85
4,Valle d'Aosta,7,27,34,20.59
5,Piemonte,127,501,628,20.22
6,Abruzzo,36,145,181,19.89
7,Toscana,112,458,570,19.65
8,Emilia-Romagna,149,740,889,16.76
9,Sicilia,141,707,848,16.63


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))